In [ ]:
%cd ../../../

In [2]:
import os
import numpy as np
import pickle

import smart_settings

from mbrl.environments import env_from_string

In [ ]:
# Needed for the recording only if LD_PRELOAD is not unset!
from mujoco_py import GlfwContext
GlfwContext(offscreen=True)

In [3]:
import imageio

def setup_video(output_path, name_suffix, name_prefix, fps):
    os.makedirs(output_path, exist_ok=True)
    file_path = os.path.join(output_path, f"{name_prefix}rollout{name_suffix}.mp4")
    i = 0
    while os.path.isfile(file_path):
        i += 1
        file_path = os.path.join(output_path, f"{name_prefix}rollout{name_suffix}_{i}.mp4")
    print("Record video in {}".format(file_path))
    return (
        imageio.get_writer(file_path, fps=fps, codec="h264", quality=10, pixelformat="yuv420p"), #yuv420p, yuvj422p
        file_path,
    )


In [99]:
env_name = "construction" # construction or "playground"

In [100]:
video_dir = ""
output_path = os.path.join(video_dir, "video")

## Choose if to replay free play data

In [101]:
working_dir = f'results/cee_us/{env_name}/gnn_ensemble_cee_us_freeplay'

# Specify which training iteration of cee-us to replay
iteration = 50
mode = "free_play"

## or rollouts from zero-shot downstream task generalization

In [103]:
task = "flip"    
# [stack, throw, pp, flip] for construction
# e.g. push4, push6 etc. for playground

working_dir = f'results/cee_us/zero_shot/{env_name}_{task}/gnn_ensemble_icem'

# For downstream tasks there is only 1 iteration of rollout collection
iteration = 0
mode = f"{env_name}_{task}"

print("Current mode: ", mode)

Current mode:  construction_flip


## Read settings file and initialize the corresponding environment

In [104]:
params = smart_settings.load(os.path.join(working_dir, 'settings.json'), make_immutable=False)
env = env_from_string(params.env, **params["env_params"])

In [106]:
env.nObj

4

## Load rollout buffer

In [107]:
with open(os.path.join(working_dir, 'checkpoints_latest/rollouts'), 'rb') as f:
    buffer = pickle.load(f)
    
print("Buffer contains {} rollouts, each with length {}".format(len(buffer), buffer[0]["observations"].shape[0]))

Buffer contains 100 rollouts, each with length 500


## Record video

In [ ]:
video, video_path = setup_video(output_path, f'_{mode}_iter{iteration}', "", env.get_fps())

ep_length = buffer[0]["observations"].shape[0]

render_width = 768
render_height = 512

ep_id = iteration * params.number_of_rollouts 
for i in range(ep_id, min(ep_id + params.number_of_rollouts, len(buffer))):
    obs = env.reset()
    for t in range(ep_length):
        if env.name == "PlaygroundwGoals":
            env.set_state_from_observation(buffer[i]["observations"][t, :])
        else:
            # Robotics environments require setting the whole env state for replay!
            env.set_GT_state(buffer[i]["env_states"][t, :])
        frame = env.render("rgb_array", render_width, render_height)
        video.append_data(frame)
        if env.name == "FetchPickAndPlaceConstruction":
            del env.viewer._markers[:]

video.close()

# Render rollouts with openglw

You need to set the following variable to render: 
```bash
export LD_PRELOAD=/usr/lib/x86_64-linux-gnu/libGLEW.so
```

In [ ]:
ep_id = iteration * params.number_of_rollouts 
success_rate = []
for i in range(ep_id, min(ep_id + params.number_of_rollouts, len(buffer))):
    episode_length = buffer[i]["observations"].shape[0]
    for t in range(episode_length):
        if env.name == "PlaygroundwGoals":
            env.set_state_from_observation(buffer[i]["observations"][t, :])
        else:
            # Robotics environments require setting the whole env state for replay!
            env.set_GT_state(buffer[i]["env_states"][t, :])
        env.render()    
env.close()

In [108]:
if mode != "free_play":
    success_rate = []
    for i in range(len(buffer)):
        rollout_success = env.eval_success(buffer[i]["next_observations"])
        stable_T = 5
        if env.name == "FetchPickAndPlaceConstruction" and "tower" in env.case:
            # Stack is only successful if we have a full tower! 
            # Check if the tower is stable for at least 5 timesteps
            dy = np.diff(rollout_success)
            success = np.logical_and(rollout_success[1:]==env.num_blocks, dy==0)
            success_rate.append(np.sum(success)>stable_T)
        elif env.name == "FetchPickAndPlaceConstruction" and env.case == 'PickAndPlace':
            # We determine success as highest number of solved elements with at least 5 timesteps of success
            u, c = np.unique(rollout_success, return_counts = True)
            # u: unique values, c: counts
            count_of_highest_success = c[np.argmax(u)]    
            success_rate.append(u[c>stable_T][-1]/env.nObj)
        else:
            # For flip, throw and Playground env push tasks: just get success at the end of rollout
            success_rate.append(rollout_success[-1]/env.nObj)
    print("Success rate over {} rollouts is {}".format(len(buffer), np.asarray(success_rate).mean()))

Success rate over 100 rollouts is 0.92
